In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from package.databases.initialize import initialize_memories
initialize_memories()

🔁 Trigger already exists, skipping.


In [3]:
# from sqlmodel import Session
# from sqlalchemy import text
# from your_app.db import engine  # adjust as needed

# with Session(engine) as session:
#     # Enable extensions
#     session.exec(text("CREATE EXTENSION IF NOT EXISTS pg_trgm;"))
#     session.exec(text("CREATE EXTENSION IF NOT EXISTS unaccent;"))

#     # Optional: check for existence of the built-in trigger function
#     result = session.exec(
#         text("SELECT proname FROM pg_proc WHERE proname = 'tsvector_update_trigger';")
#     ).all()

#     if not result:
#         print("⚠️ tsvector_update_trigger not found!")
#     else:
#         print("✅ tsvector_update_trigger is available.")


In [4]:
# from package.databases.models.jargon import Jargon
# from sqlmodel import Session
# from package.databases.engine import engine

# with Session(engine) as session:
#     j = Jargon(
#         jargon="neural architecture",
#         evidence="Deep learning uses this",
#         explanation="An arrangement of artificial neurons",
#         document_id='ab2900cc-0b68-4eda-837e-ad818069e403'
#     )
#     session.add(j)
#     session.commit()

In [5]:
# from sqlalchemy import select, func
# from sqlmodel import Session

# def search_jargon(term: str):
#     ts_query = func.to_tsquery("english", term)
#     with Session(engine) as session:
#         stmt = select(Jargon).where(Jargon.search_vector.op('@@')(ts_query))
#         results = session.exec(stmt).all()
#         return results


# results = search_jargon("neural & architecture")
# results


In [6]:
from package.utils.data_loder import PDFLoader
from package.interface import SourceOptions
from package.flows.offline import OfflineFlow
from package.databases.management.longterm import LongTermManagement
from package.databases.management.user import UserManagement
from package.databases.management.document import DocumentManagement
from package.databases.management.project import ProjectManagement
from package.databases.management.jargon import JargonManagement
from package.databases.session import Session, get_session, Depends
from package.databases.models.user import User
from package.databases.models.document import Document
from package.databases.models.project import Project
from package.databases.models.jargon import Jargon
from package.databases.models.longterm import LongTerm

d:\broai-arai\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
user = User(
    username="bank",
    password="555",
    email="bank@bank.com"
)

um = UserManagement()
user = um.create_user(user, session=Depends(get_session))

In [8]:
user.id

'9157bd9f-7eda-4fd9-bf0b-c53526984560'

In [9]:
document1 = Document(source="./sources/storm.pdf", type="pdf")

dm = DocumentManagement()
document1 = dm.create_document(document1, session=Depends(get_session)) 

In [10]:
document1.id

'97bad39a-538e-4135-b87d-88c7252b1962'

In [11]:
dm.read_document_longterms(document_id=document1.id, session=Depends(get_session))

[]

In [12]:
source_ops = SourceOptions(path="./sources/storm.pdf", type="pdf")
loader = PDFLoader(
    source=source_ops
)
contexts = loader.run()

Markdown headings: max(2)


d:\broai-arai\backend\package\utils\data_loder.py:22: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(text)
d:\broai-arai\backend\package\utils\data_loder.py:23: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)
d:\broai-arai\backend\package\utils\data_loder.py:24: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)
d:\broai-arai\backend\package\utils\data_loder.py:30: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts, max_tokens=max_tokens, overlap=overlap)


In [13]:
olf = OfflineFlow()

In [14]:
longterms = olf.run(document_id=document1.id, contexts=contexts)

In [15]:
ltm = LongTermManagement()
ltm.create_raws(longterms, session=Depends(get_session))

In [16]:
# from package.embedding.baai import BAAIEmbedding
# embedding = BAAIEmbedding()

In [17]:
document1.id

'97bad39a-538e-4135-b87d-88c7252b1962'

In [18]:
longterms = ltm.read_longterms_by_document(document_id=document1.id, session=Depends(get_session))

In [19]:
# sentences = [longterm.raw for longterm in longterms]
# vectors = embedding.run(sentences=sentences)
# for longterm, vector in zip(longterms, vectors):
#     longterm.raw_embedding = vector
# ltm.update_longterms(longterms=longterms, session=Depends(get_session))

In [20]:
# # query = """What does ROUGE Score do?"""
# query = """How was FreshWiki created?"""
# vector = embedding.run(sentences=[query])[0]
# results = ltm.read_similar_text(vector, limit=15, embed_method="raw", sources=[document1.source], session=Depends(get_session))

In [21]:
# for result in results:
#     print(result.meta)

In [22]:
document_chunks = [{"document_id": document1.id, "longterm_id": longterm.id} for longterm in longterms]

In [23]:
len(document_chunks)

87

In [24]:
document_chunks[0]

{'document_id': '97bad39a-538e-4135-b87d-88c7252b1962',
 'longterm_id': '654998b2-2e99-4c8b-b1e0-79acbf38801c'}

In [25]:
import boto3
import json
import uuid

# Replace this with your actual Step Function ARN
STATE_MACHINE_ARN = "arn:aws:states:ap-southeast-1:112557628841:stateMachine:broai-arai-enrich-fleet"

# Create a unique name for this execution
execution_name = f"test-execution-{uuid.uuid4()}"

# Input payload
input_payload = {
    "document_chunks": document_chunks[:]
}

# Create Step Functions client
sfn = boto3.client("stepfunctions", region_name="ap-southeast-1")

# Start execution
response = sfn.start_execution(
    stateMachineArn=STATE_MACHINE_ARN,
    name=execution_name,
    input=json.dumps(input_payload)
)

print("Execution started!")
print("Execution ARN:", response["executionArn"])

Execution started!
Execution ARN: arn:aws:states:ap-southeast-1:112557628841:execution:broai-arai-enrich-fleet:test-execution-2214e3d3-0d0f-4053-84f2-8fed9448d23f


In [26]:
import time

execution_arn = response["executionArn"]

while True:
    desc = sfn.describe_execution(executionArn=execution_arn)
    status = desc["status"]
    print(f"Execution status: {status}")
    if status in ("SUCCEEDED", "FAILED", "TIMED_OUT", "ABORTED"):
        break
    time.sleep(2)

if status == "SUCCEEDED":
    output = desc.get("output")
    print("Execution output:", output)
else:
    print(f"Execution ended with status: {status}")

Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: SUCCEEDED
Execution output: {"status":"success","results":[{"Payload":{"processed":"654998b2-2e99-4c8b-b1e0-79acbf38801c","document_id":"97bad39a-538e-4135-b87d-88c7252b1962"}},{"Payload":{"processed":"5ff3a9b3-6f97-47ef-983d-677a7f13cdcb","document_id":"97bad39a-538e-4135-b87d-88c7252b1962"}},{"Payload":{"processed":"d3bd387b-6a12-4837-903c-8bbc76c9c2fa","document_id":"97bad39a-538e-4135-b87d-88c7252b1962"}},{"Payload":{"processed":"c6be679e-fff3-4842-b969-472c2daa2e5f","document_id":"97bad39a-538e-4135-b87d-88c7252b1962"}},{"Payload":{"processed":"3dfa1242-2bc6-4f00-a2b1-719f82e671fa","document_id":"97bad39a-538e-4135-b87d-88c7252b1962"}},{"Payload":{"processed":"21c08f0a-1758-4887-a8f7-7dfd6bed606d","document_id":"97bad39a-538e-4135-b87d-88c7252b1962"}},{"Payload":{"processed":"52b791b0-3792-4512-ad00-d6a0ccce58

In [27]:
jm = JargonManagement()
dm = DocumentManagement()

In [28]:
len(jm.read_jargons(session=Depends(get_session)))

206

In [29]:
_longterms = ltm.read_longterms_by_document(document_id=document1.id, session=Depends(get_session))
len(_longterms)

87

In [30]:
from package.databases.utils import now_utc
def embed(longterms, embedder, session: Session = Depends(get_session)):
    updated_at = now_utc()
    raws = [longterm.raw for longterm in longterms]
    raw_vectors = embedder.run(sentences=raws)
    enrichs = [longterm.enrich for longterm in longterms]
    enrich_vectors = embedder.run(sentences=enrichs)
    combos = [longterm.combo for longterm in longterms]
    combo_vectors = embedder.run(sentences=combos)
    for longterm, vector in zip(longterms, raw_vectors):
        longterm.raw_embedding = vector
    for longterm, vector in zip(longterms, enrich_vectors):
        longterm.enrich_embedding = vector
    for longterm, vector in zip(longterms, combo_vectors):
        longterm.combo_embedding = vector
        longterm.updated_at = updated_at
    ltm.update_longterms(longterms=longterms, session=session)

In [31]:
from package.embedding.baai import BAAIEmbedding
embedder = BAAIEmbedding()
embed(_longterms, embedder=embedder, session=Depends(get_session))

🔍 Loading model from: BAAI/bge-m3


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [32]:
ltm.read_longterms_by_document(document_id=document1.id, session=Depends(get_session))[0]

LongTerm(raw="2 FreshWiki\n\nWe study generating Wikipedia-like articles from scratch, placing emphasis on the pre-writing stage (Rohman, 1965), which involves the demanding sub-tasks of gathering and curating relevant information ('research'). This models the human\n\n1 Our resources and code are released at https://github. com/stanford-oval/storm .\n\nTable 1: Comparison of different Wikipedia generation setups in existing literature. Generating one paragraph does not need an article outline.\n\n|                            | Domain   | Scope        | Given Outline?   | Given Refs?   |\n|----------------------------|----------|--------------|------------------|---------------|\n| Balepur et al. (2023)      | One      | One para.    | /                | Yes           |\n| Qian et al. (2023)         | All      | One para.    | /                | No            |\n| Fan and Gardent (2022)     | One      | Full article | Yes              | No            |\n| Liu et al. (2018)          | A

In [33]:
vector = embedder.run(sentences=["How was FreshWiki created?"])[0]
response = ltm.read_similar_text(vector=vector, embed_method="raw", session=Depends(get_session))
for r in response:
    print(r.meta)

{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '2 FreshWiki', 'sequence': 5}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '2.1 The FreshWiki Dataset', 'sequence': 7}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'A Dataset Details', 'sequence': 40}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '8 Conclusion', 'sequence': 29}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'Abstract', 'sequence': 2}


In [34]:
# vector = embedder.run(sentences=["How was FreshWiki created?"])[0]
response = ltm.read_similar_text(vector=vector, embed_method="enrich", session=Depends(get_session))
for r in response:
    print(r.meta)

{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '2.1 The FreshWiki Dataset', 'sequence': 7}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'A Dataset Details', 'sequence': 40}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '4 Experiments', 'sequence': 14}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '## Assisting in Writing Wikipedia-like Articles From Scratch with Large Language Models', 'sequence': 0}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '2 FreshWiki', 'sequence': 5}


In [35]:
# vector = embedder.run(sentences=["How was FreshWiki created?"])[0]
response = ltm.read_similar_text(vector=vector, embed_method="combo", session=Depends(get_session))
for r in response:
    print(r.meta)

{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '2.1 The FreshWiki Dataset', 'sequence': 7}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': 'A Dataset Details', 'sequence': 40}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '2 FreshWiki', 'sequence': 5}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '8 Conclusion', 'sequence': 29}
{'type': 'pdf', 'source': './sources/storm.pdf', 'section': '4 Experiments', 'sequence': 14}


In [64]:
from package.databases.engine import engine
from sqlalchemy import select, func, or_
from sqlmodel import Session

def search_jargon(term: str):
    ts_query = func.to_tsquery("english", term)
    with Session(engine) as session:
        stmt = select(Jargon).where(
            or_(
                Jargon.search_vector.op('@@')(ts_query),
                func.similarity(Jargon.jargon, term) > 0.3
                
            )
            )
        results = session.exec(stmt).all()
        return results


results = search_jargon("STARM")
len(results)

26

In [71]:
results[5][0].evidence

'We propose STORM , a writing system for the S ynthesis of T opic O utlines through R etrieval and M ulti-perspective Question Asking'

In [72]:
texts = [result[0].evidence for result in results]

In [75]:
print("\n".join(texts))

STORM simulates a conversation between a Wikipedia writer and a topic expert
STORM is capable of researching complicated topics and writing long articles from detailed outlines
STORM creates an outline before the actual writing starts
Given the input topic t , STORM discovers different perspectives by surveying existing articles from similar topics
We present STORM to automate the pre-writing stage
We propose STORM , a writing system for the S ynthesis of T opic O utlines through R etrieval and M ulti-perspective Question Asking
Algorithm 1: STORM
we introduce STORM, a framework that automates the pre-writing stage
articles produced by STORM
STORM prompts LLMs to ask effective questions by discovering specific perspectives and simulating multi-turn conversations
we propose the STORM paradigm for the S ynthesis of T opic O utlines through R etrieval and M ulti-perspective Question Asking
STORM             | 45.82
4.4 STORM Implementation
STORM
articles produced by STORM
We propose STORM

In [13]:
from package.databases.destroy import drop_all_tables

drop_all_tables()

✅ All tables dropped.
